# Importar Bibliotecas que serão utilizadas

In [1]:
import xlsxwriter
import pandas as pd
import numpy as np
import re
from pandas import DataFrame

import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

import warnings
from pandas_ui import *
import sweetviz as sv

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

# Fazer a leitura do arquivo dispensando as linhas do cabeçalho

In [2]:
data = pd.read_excel('soja_milho.xlsx',sheet_name='precos', skiprows = 7)

In [3]:
data

,Observação,Data,Mês,Ano - Safra,Preços - Milho,Preços - Soja
0,1,02/01/2014,1,2014,26.64,72.18
1,2,03/01/2014,1,2014,27.00,71.42
2,3,06/01/2014,1,2014,27.29,71.38
3,4,07/01/2014,1,2014,27.49,70.31
4,5,08/01/2014,1,2014,27.16,70.14
...,...,...,...,...,...,...
493,494,22/12/2015,12,2015,36.23,78.61
494,495,23/12/2015,12,2015,36.26,78.21
495,496,28/12/2015,12,2015,36.29,77.59
496,497,29/12/2015,12,2015,36.55,78.55


# Análise manual

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Observação      498 non-null    int64  
 1   Data            498 non-null    object 
 2   Mês             498 non-null    int64  
 3   Ano - Safra     498 non-null    int64  
 4   Preços - Milho  498 non-null    float64
 5   Preços - Soja   498 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 23.5+ KB


In [5]:
data.head()

,Observação,Data,Mês,Ano - Safra,Preços - Milho,Preços - Soja
0,1,02/01/2014,1,2014,26.64,72.18
1,2,03/01/2014,1,2014,27.00,71.42
2,3,06/01/2014,1,2014,27.29,71.38
3,4,07/01/2014,1,2014,27.49,70.31
4,5,08/01/2014,1,2014,27.16,70.14


In [6]:
data.describe()

,Observação,Mês,Ano - Safra,Preços - Milho,Preços - Soja
count,498.000000,498.000000,498.000000,498.000000,498.000000
mean,249.500000,6.528112,2014.497992,27.955080,66.498273
std,143.904482,3.433211,0.500499,3.563466,5.804226
min,1.000000,1.000000,2014.000000,21.760000,56.740000
25%,125.250000,4.000000,2014.000000,25.510000,62.302500
50%,249.500000,7.000000,2014.000000,27.535000,65.165000
75%,373.750000,9.000000,2015.000000,30.320000,69.187500
max,498.000000,12.000000,2015.000000,36.830000,79.860000


In [7]:
data.mode()

,Observação,Data,Mês,Ano - Safra,Preços - Milho,Preços - Soja
0,1,01/04/2014,7.0,2014.0,25.16,62.02
1,2,01/04/2015,NaN,NaN,27.60,62.29
2,3,01/06/2015,NaN,NaN,33.82,67.52
3,4,01/07/2014,NaN,NaN,NaN,NaN
4,5,01/07/2015,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
493,494,31/03/2015,NaN,NaN,NaN,NaN
494,495,31/07/2014,NaN,NaN,NaN,NaN
495,496,31/07/2015,NaN,NaN,NaN,NaN
496,497,31/08/2015,NaN,NaN,NaN,NaN


     Observação      Qualitativo  
     Data            Qualitativo 
     Mês             Qualitativo 
     Ano - Safra     Qualitativo 
     Preços - Milho  Quantitativo Contínuo
     Preços - Soja   Quantitativo Contínuo

  
                          Qualitativo           Qualitativo Qualitativo     Quantitativo Contínui
                          Observação         	Mês      	Ano - Safra    Preços - Milho   Preços - Soja
         Média =    	  249.500000 	        6.528112    2014.497992     27.955080        66.498273
        Mediana =         249.500000         	7.000000	2014.000000 	27.535000    	65.165000
         Moda =                                  7.0         2014.0          33.82        	67.52
     Quartis  
          25% =           125.250000 	        4.000000 	2014.000000 	25.510000   	62.302500
          50% =	       249.500000             7.000000 	2014.000000 	27.535000   	65.165000
          75% = 	      373.750000         	9.000000 	2015.000000 	30.320000   	69.187500
          max =           498.000000 	        12.000000    2015.000000     36.830000       79.860000


In [8]:
boxplot = data.boxplot(column=['Preços - Milho',   'Preços - Soja'])

# Não foi encontrado outlines

In [9]:
plt.scatter(data['Preços - Milho'], data['Preços - Soja'])
plt.title('Scatterplot da correlação Milho - Soja')
plt.xlabel('Preço - Milho')
plt.ylabel('Preço- Soja')
plt.show()

/home/adolpho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  """


# Análise utilizando Biblioteca especializada

# Necessário trabalhar com o CSV deste arquivo no Pandas_UI

In [10]:
data.to_csv('Soja-Milho.csv')

In [11]:
pandas_ui('Soja-Milho.csv')

# Sweetviz é outra biblioteca especializada para análise que produz um bom report

In [12]:
sweetviz_report = sv.analyze([data,'data'],target_feat='Preços - Milho')

:FEATURES DONE:                    |█████████████████████| [100%]   00:16  -> (00:00 left)
:PAIRWISE DONE:                    |█████████████████████| [100%]   00:00  -> (00:00 left)


Creating Associations graph... DONE!


In [13]:
sweetviz_report.show_html('viz.html')

Report viz.html was generated! NOTEBOOK/COLAB USERS: no browser will pop up, the report is saved in your notebook/colab files.


In [14]:
sweetviz_report = sv.analyze([data,'data'],target_feat='Preços - Soja')

:FEATURES DONE:                    |█████████████████████| [100%]   00:16  -> (00:00 left)
:PAIRWISE DONE:                    |█████████████████████| [100%]   00:00  -> (00:00 left)


Creating Associations graph... DONE!


In [15]:
sweetviz_report.show_html('viz.html')

Report viz.html was generated! NOTEBOOK/COLAB USERS: no browser will pop up, the report is saved in your notebook/colab files.


# Podemos visualizar bem a correlação dos preços

In [16]:
a = np.corrcoef(data['Preços - Milho'], data['Preços - Soja'])
print('A matriz correlação de milho e soja é:\n\n', a)

A matriz correlação de milho e soja é:

 [[1.         0.74909511]
 [0.74909511 1.        ]]


In [17]:
print('A correlação de milho e soja é: ', a[0,1])

A correlação de milho e soja é:  0.749095112208888


In [18]:
print('A matriz correlação de toda a planilha é:\n\n', data.corr())

A matriz correlação de toda a planilha é:

                 Observação       Mês  Ano - Safra  Preços - Milho  \
Observação        1.000000  0.496192     0.866020        0.295683   
Mês               0.496192  1.000000    -0.002309        0.054193   
Ano - Safra       0.866020 -0.002309     1.000000        0.305910   
Preços - Milho    0.295683  0.054193     0.305910        1.000000   
Preços - Soja     0.432048  0.316132     0.315516        0.749095   

                Preços - Soja  
Observação           0.432048  
Mês                  0.316132  
Ano - Safra          0.315516  
Preços - Milho       0.749095  
Preços - Soja        1.000000  


In [19]:
b = np.cov(data['Preços - Milho'], data['Preços - Soja'])
print('A matriz de covariância é:\n\n', b)

A matriz de covariância é:

 [[12.69829305 15.49365789]
 [15.49365789 33.68904128]]


In [20]:
print('A covariância de milho e soja é: ',b[0,1])

A covariância de milho e soja é:  15.493657885465405


### Analisando a covariância vemos uma relação diretamente proporcional entre o milho e a soja com o valor positivo, e a covariância percebemos uma corelação forte entre o milho e a soja, sendo necessário o valor estar entre 0.7 e 0.89 para a correlação forte.

### Analisando os dois reports com target diferente, vemos o comportamento semelhante em relação ao ano - safra devido a sua correlação antes já percebida

# Construir um classificador que se enquadre perfeitamente a uma função

### Criar a Coluna Y com os valores reais do classificador. Os valores reais corresponderão as seguintes regras

- Caso ((ano é 2014) E (preço ≥ 67,00 reais) Então é Colheita)
- Caso ((ano é 2015) E (58,00 reais ≤ preço≤ 62,00 reais ) Então é Colheita)
- Caso contrário é Plantio.

In [21]:
data = data.rename(columns={'Mês': 'Mes',
                            'Ano - Safra': 'AnoSafra',
                            'Preços - Milho':'PreçosMilho',
                            'Preços - Soja': 'PreçosSoja'}, inplace = False)
data['Y'] = 0

In [22]:
data.head()

,Observação,Data,Mes,AnoSafra,PreçosMilho,PreçosSoja,Y
0,1,02/01/2014,1,2014,26.64,72.18,0
1,2,03/01/2014,1,2014,27.00,71.42,0
2,3,06/01/2014,1,2014,27.29,71.38,0
3,4,07/01/2014,1,2014,27.49,70.31,0
4,5,08/01/2014,1,2014,27.16,70.14,0


In [23]:
for i in data.itertuples():
    if(i.Mes <=5):
        if (i.AnoSafra == 2014 and i.PreçosSoja >= float (67.00)):
            data.Y[i.Index] = 1
        else:
            if (i.AnoSafra == 2015 and float (58.00) <= i.PreçosSoja <= float (62.00)):
                data.Y[i.Index] = 1
            else:
                data.Y[i.Index] = 0
    else:
        data.Y[i.Index] = 0
        
warnings.filterwarnings('ignore')

/home/adolpho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/adolpho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/adolpho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Th

In [24]:
data.describe()

,Observação,Mes,AnoSafra,PreçosMilho,PreçosSoja,Y
count,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000
mean,249.500000,6.528112,2014.497992,27.955080,66.498273,0.238956
std,143.904482,3.433211,0.500499,3.563466,5.804226,0.426875
min,1.000000,1.000000,2014.000000,21.760000,56.740000,0.000000
25%,125.250000,4.000000,2014.000000,25.510000,62.302500,0.000000
50%,249.500000,7.000000,2014.000000,27.535000,65.165000,0.000000
75%,373.750000,9.000000,2015.000000,30.320000,69.187500,0.000000
max,498.000000,12.000000,2015.000000,36.830000,79.860000,1.000000


## A maioria dos valores de y, 76,11%, correspondem ao valor 0, que é plantio

In [25]:
plantio_index = (data['Y'] == 0)
colheita_index = (data['Y'] == 1)
plantio = data[plantio_index]
colheita = data[colheita_index]

In [26]:
plantio.describe()

,Observação,Mes,AnoSafra,PreçosMilho,PreçosSoja,Y
count,379.000000,379.000000,379.000000,379.000000,379.000000,379.0
mean,282.496042,7.630607,2014.538259,27.549103,66.777916,0.0
std,134.716484,3.121137,0.499193,3.729992,6.237840,0.0
min,14.000000,1.000000,2014.000000,21.760000,56.740000,0.0
25%,170.500000,6.000000,2014.000000,25.000000,62.460000,0.0
50%,295.000000,8.000000,2015.000000,26.930000,64.540000,0.0
75%,403.500000,10.000000,2015.000000,29.585000,71.710000,0.0
max,498.000000,12.000000,2015.000000,36.830000,79.860000,0.0


In [27]:
colheita.describe()

,Observação,Mes,AnoSafra,PreçosMilho,PreçosSoja,Y
count,119.000000,119.000000,119.000000,119.000000,119.000000,119.0
mean,144.411765,3.016807,2014.369748,29.248067,65.607647,1.0
std,120.210508,1.449624,0.484778,2.588414,4.024500,0.0
min,1.000000,1.000000,2014.000000,24.970000,58.170000,1.0
25%,48.500000,2.000000,2014.000000,27.310000,61.400000,1.0
50%,87.000000,3.000000,2014.000000,28.720000,67.530000,1.0
75%,276.500000,4.000000,2015.000000,31.330000,68.650000,1.0
max,348.000000,5.000000,2015.000000,34.160000,72.180000,1.0


### Temos, portanto, 379 observações de plantio e 119 observações de colheita

### Para melhorar os resultados do classificador, irei utilizar Oversampling nos dados de treino
- Apesar de usar a coluna mês para criar o algorítmo real, ela será retirada na hora de fazer o algorítmo de ML para analisar o comportamento sem a coluna

In [28]:
X = data.copy()

In [29]:
X.pop('Observação')
X.pop('Data')
X.pop('PreçosMilho')
X.pop('Mes')
X.pop('Y')
y = data['Y']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 77)

In [31]:
sm = SMOTE(random_state=77, sampling_strategy = 1.0)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

# Devido a dinâmica da função, com algoritmo do estilo função if e else, irei utilizar o algorítmo Random Forest.
# Ambos algorítmos são capazes de criar cortes nos eixos X e Y da função resposta.

In [32]:
clf_rf = RandomForestClassifier(n_estimators=25, random_state=77)
clf_rf.fit(X_train_res, y_train_res)

RandomForestClassifier(n_estimators=25, random_state=77)

In [33]:
y_res = clf_rf.predict(X_test)

In [34]:
print(pd.crosstab(y_test, y_res, rownames=['Real'], colnames=['Predicted'], margins=True))

Predicted    0   1  All
Real                   
0          108   4  112
1            3  35   38
All        111  39  150


In [35]:
acc = accuracy_score(y_test, y_res)
acc

0.9533333333333334

In [36]:
prec = precision_score(y_test, y_res)
prec

0.8974358974358975

In [37]:
especificidade = precision_score(y_test, y_res, pos_label = 0)
especificidade

0.972972972972973

In [38]:
recall = recall_score(y_test, y_res)
recall

0.9210526315789473

In [39]:
f1 = f1_score(y_test, y_res)
f1

0.9090909090909091

In [40]:
classification_report(y_test, y_res)

'              precision    recall  f1-score   support\n\n           0       0.97      0.96      0.97       112\n           1       0.90      0.92      0.91        38\n\n    accuracy                           0.95       150\n   macro avg       0.94      0.94      0.94       150\nweighted avg       0.95      0.95      0.95       150\n'

In [41]:
auc = roc_auc_score(y_test, y_res)
auc

0.9426691729323309

In [42]:
y_prob = clf_rf.predict_proba(X_test)

In [43]:
# Analisar os valores que irão compor a curva ROC

y_prob

array([[0.   , 1.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [0.   , 1.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [0.48 , 0.52 ],
       [0.   , 1.   ],
       [0.04 , 0.96 ],
       [0.   , 1.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [0.   , 1.   ],
       [0.   , 1.   ],
       [0.622, 0.378],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [0.   , 1.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [0.96 , 0.04 ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [1.   , 0.   ],
       [0.96 , 0.04 ],
       [0.2

In [44]:
probs = y_prob[:,1]
rfp, rvp, lim = roc_curve(y_test, probs)

In [45]:
probs = clf_rf.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

In [46]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'r', label = 'AUC = %0.2f' % auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'g--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()